# ConversationBufferMemory

- Author: [Sungchul Kim](https://github.com/rlatjcj)
- Design: [Sungchul Kim](https://github.com/rlatjcj)
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-4/sub-graph.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239937-lesson-2-sub-graphs)

## Overview

This tutorial introduces the `ConversationBufferMemory` class, which is a memory class that stores conversation history in a buffer.  
Usually, it doesn't need any additional processing, but sometimes it may be required, such as when the conversation history is too large to fit in the context window of the model.  
In this tutorial, we will learn how to use the `ConversationBufferMemory` class to store and retrieve conversation history.


### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Extract messages as strings](#extract-messages-as-strings)
- [Extract messages as `HumanMessage` and `AIMessage` objects](#extract-messages-as-humanmessage-and-aimessage-objects)
- [Apply to Chain](#apply-to-chain)

### References
- [LangChain: ConversationBufferMemory](https://python.langchain.com/api_reference/langchain/memory/langchain.memory.buffer.ConversationBufferMemory.html)
----

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
!pip install langchain-opentutorial

In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_openai",
    ],
    verbose=False,
    upgrade=False,
)

In [4]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "ConversationBufferMemory",
    }
)

Environment variables have been set successfully.


You can alternatively set `OPENAI_API_KEY` in `.env` file and load it.

**[Note]**  
- This is not necessary if you've already set `OPENAI_API_KEY` in previous steps.

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

## Extract messages as strings

This memory allows you to extract messages into a variable after storing them.


In [5]:
from langchain.memory import ConversationBufferMemory

In [6]:
memory = ConversationBufferMemory()
memory

/tmp/ipykernel_188575/2223904900.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]))

You can use the `save_context(inputs, outputs)` method to save conversation records.

- This method takes two arguments, `inputs` and `outputs`.
- `inputs` stores the user's input, and `outputs` stores the AI's output.
- Using this method saves the conversation record under the `history` key.
- Later, you can use the `load_memory_variables` method to check the saved conversation record.


In [7]:
# inputs: dictionary(key: "human" or "ai", value: question)
# outputs: dictionary(key: "ai" or "human", value: answer)
memory.save_context(
    inputs={
        "human": "Hello, I want to open a bank account remotely. How do I start?",
    },
    outputs={
        "ai": "Hello! I'm glad you want to open an account. First, please prepare your ID for identity verification."
    },
)

In [12]:
memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Hello, I want to open a bank account remotely. How do I start?', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello! I'm glad you want to open an account. First, please prepare your ID for identity verification.", additional_kwargs={}, response_metadata={})]))

The `load_memory_variables({})` function of memory returns the message history.

In [13]:
# Check the message history stored in the 'history' key.
print(memory.load_memory_variables({})["history"])

Human: Hello, I want to open a bank account remotely. How do I start?
AI: Hello! I'm glad you want to open an account. First, please prepare your ID for identity verification.


In [14]:
memory.save_context(
    inputs={
        "human": "Yes, I've prepared my ID for identity verification. What should I do next?"
    },
    outputs={
        "ai": "Thank you. Please upload the front and back of your ID clearly. We will proceed with the identity verification process next."
    },
)

In [15]:
# Check the message history stored in the 'history' key.
print(memory.load_memory_variables({})["history"])

Human: Hello, I want to open a bank account remotely. How do I start?
AI: Hello! I'm glad you want to open an account. First, please prepare your ID for identity verification.
Human: Yes, I've prepared my ID for identity verification. What should I do next?
AI: Thank you. Please upload the front and back of your ID clearly. We will proceed with the identity verification process next.


In [16]:
# Save 2 conversations.
memory.save_context(
    inputs={
        "human": "I uploaded the photo. How do I proceed with identity verification?"
    },
    outputs={
        "ai": "We have confirmed the photo you uploaded. Please proceed with identity verification through your mobile phone. Please enter the verification number sent by text."
    },
)
memory.save_context(
    inputs={
        "human": "I entered the verification number. How do I open an account now?"
    },
    outputs={
        "ai": "Identity verification has been completed. Please select the type of account you want and enter the necessary information. You can choose the type of deposit, currency, etc."
    },
)

In [17]:
# Check the conversation history stored in the 'history' key.
print(memory.load_memory_variables({})["history"])

Human: Hello, I want to open a bank account remotely. How do I start?
AI: Hello! I'm glad you want to open an account. First, please prepare your ID for identity verification.
Human: Yes, I've prepared my ID for identity verification. What should I do next?
AI: Thank you. Please upload the front and back of your ID clearly. We will proceed with the identity verification process next.
Human: I uploaded the photo. How do I proceed with identity verification?
AI: We have confirmed the photo you uploaded. Please proceed with identity verification through your mobile phone. Please enter the verification number sent by text.
Human: I entered the verification number. How do I open an account now?
AI: Identity verification has been completed. Please select the type of account you want and enter the necessary information. You can choose the type of deposit, currency, etc.


In [18]:
# Save 2 more conversations.
memory.save_context(
    inputs={
        "human": "I've entered all the information. What's the next step?",
    },
    outputs={
        "ai": "I've confirmed the information you've entered. The account opening process is almost complete. Please agree to the terms of use and confirm the account opening."
    },
)
memory.save_context(
    inputs={
        "human": "I've completed all the steps. Has the account been opened?",
    },
    outputs={
        "ai": "Yes, the account has been opened. Your account number and related information have been sent to the email you registered. If you need additional help, please contact us at any time. Thank you!"
    },
)

In [19]:
# Check the conversation history stored in the 'history' key.
print(memory.load_memory_variables({})["history"])

Human: Hello, I want to open a bank account remotely. How do I start?
AI: Hello! I'm glad you want to open an account. First, please prepare your ID for identity verification.
Human: Yes, I've prepared my ID for identity verification. What should I do next?
AI: Thank you. Please upload the front and back of your ID clearly. We will proceed with the identity verification process next.
Human: I uploaded the photo. How do I proceed with identity verification?
AI: We have confirmed the photo you uploaded. Please proceed with identity verification through your mobile phone. Please enter the verification number sent by text.
Human: I entered the verification number. How do I open an account now?
AI: Identity verification has been completed. Please select the type of account you want and enter the necessary information. You can choose the type of deposit, currency, etc.
Human: I've entered all the information. What's the next step?
AI: I've confirmed the information you've entered. The accoun

## Extract messages as `HumanMessage` and `AIMessage` objects

Setting `return_messages=True` returns `HumanMessage` and `AIMessage` objects.


In [20]:
memory = ConversationBufferMemory(return_messages=True)

memory.save_context(
    inputs={
        "human": "Hello, I want to open a bank account remotely. How do I start?",
    },
    outputs={
        "ai": "Hello! I'm glad you want to open an account. First, please prepare your ID for identity verification.",
    },
)

memory.save_context(
    inputs={
        "human": "Yes, I've prepared my ID for identity verification. What should I do next?"
    },
    outputs={
        "ai": "Thank you. Please upload the front and back of your ID clearly. We will proceed with the identity verification process next."
    },
)

memory.save_context(
    inputs={
        "human": "I uploaded the photo. How do I proceed with identity verification?"
    },
    outputs={
        "ai": "We have confirmed the photo you uploaded. Please proceed with identity verification through your mobile phone. Please enter the verification number sent by text."
    },
)

In [21]:
# Check the conversation history stored in the 'history' key.
memory.load_memory_variables({})["history"]

[HumanMessage(content='Hello, I want to open a bank account remotely. How do I start?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello! I'm glad you want to open an account. First, please prepare your ID for identity verification.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="Yes, I've prepared my ID for identity verification. What should I do next?", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Thank you. Please upload the front and back of your ID clearly. We will proceed with the identity verification process next.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I uploaded the photo. How do I proceed with identity verification?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='We have confirmed the photo you uploaded. Please proceed with identity verification through your mobile phone. Please enter the verification number sent by text.', additional_kwargs={}, response_metadata=

## Apply to Chain

Let's apply the `ConversationBufferMemory` to the `ConversationChain`.

In [23]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain

# Create an LLM model.
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

# Create a ConversationChain.
conversation = ConversationChain(
    # Use ConversationBufferMemory.
    llm=llm,
    memory=ConversationBufferMemory(),
)

/tmp/ipykernel_188575/3549519840.py:8: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(


Proceed with the conversation using the `ConversationChain`.

In [25]:
# Start the conversation.
response = conversation.predict(
    input="Hello, I want to open a bank account remotely. How do I start?"
)
print(response)

Hello again! Opening a bank account remotely is a convenient process, and I’m happy to guide you through it. Here’s a detailed step-by-step guide to help you get started:

1. **Research and Choose a Bank**: Start by researching different banks to find one that suits your needs. Consider factors like account fees, interest rates, customer service, and any special features they offer. Online reviews and comparison websites can be helpful in making your decision.

2. **Visit the Bank’s Website**: Once you’ve chosen a bank, head to their official website. Look for the section dedicated to personal banking or account opening. Most banks have a clear pathway for opening accounts online.

3. **Select the Type of Account**: Decide on the type of account you want to open. Common options include checking accounts, savings accounts, or a combination of both. Some banks also offer specialized accounts like student accounts or high-yield savings accounts.

4. **Prepare Required Documents**: Gather 

Check if the previous conversation history is being remembered.

In [26]:
# Send a request to summarize the previous conversation in bullet points.
response = conversation.predict(input="Summarize the previous answer in bullet points.")
print(response)

Certainly! Here’s a summarized version of the steps to open a bank account remotely:

- **Research and Choose a Bank**: Consider fees, interest rates, customer service, and special features.
- **Visit the Bank’s Website**: Navigate to the personal banking or account opening section.
- **Select the Type of Account**: Choose between checking, savings, or specialized accounts.
- **Prepare Required Documents**: Gather ID, Social Security Number, proof of address, and income information.
- **Fill Out the Application**: Enter personal details and agree to terms and conditions.
- **Verify Your Identity**: Upload ID, answer security questions, or join a video call.
- **Fund Your Account**: Make an initial deposit via transfer, credit card, or check.
- **Receive Confirmation**: Get account number and online banking details.
- **Set Up Online Banking**: Manage your account online for transactions and bill payments.
